In [ ]:
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

reference  
'밑바닥부터 시작하는 딥러닝'의 오차역전파법 부분 참고

## Backpropagation 오차역전파

인공신경망은 일반적으로 입력부터 출력까지 좌측에서 우측으로 진행 방향을 가짐  
훈련을 위해 이와 반대방향으로 진행하는 것을 오차역전파라 함  
이때,역방향이므로 노드 간 미분값을 전달함.  
(순전파를 수행했을 때의 입력으로 들어온 값에 대해 입력 이후 계산 결과값이 얼마나 변화하는지에 대한 값)  
컴퓨터가 예측값을 높이기 위해 출력값과 실제 예측하고자 하는 값을 비교하여 가중치를 변경하는 과정이다.  

>오차역전파 과정
1. 주어진 입력값에 상관없이, 임의의 초기 가증치를 은닉층에 넣어 결과 계산  
2. 위의 결과와 실제 예측하고자 하는 값 사이의 오차 계산  
3. 가중치 업데이트
4. 오차가 줄어들지 않을때까지 1~3번 반복


역전파 구현  
한 박스에 100원 하는 사과박스를 2개 구입하고 소비세로 10%를 추가로 내야할 때의 연산

In [1]:
import numpy as np

In [2]:
class MulLayer: # 곱셈 계층
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        return x * y
    
    def backward(self, d_out):
        # 순전파 시 입력을 서로 바꾸어서 곱해줌!
        dx = d_out * self.y
        dy = d_out * self.x
        return dx, dy
    
apple_box = 100
apple_box_num = 2
tax = 1.1

# 곱셈계층
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파 수행
apple_box_price = mul_apple_layer.forward(apple_box, apple_box_num)
price = mul_tax_layer.forward(apple_box_price, tax)
print('순전파 수행 후 지불해야 할 최종 금액:', price)
print()

# 역전파 수행(순전파와 반대 순서로 호출)
d_price = 1
d_apple_box_price, d_tax = mul_tax_layer.backward(d_price)
d_apple_box, d_apple_box_num = mul_apple_layer.backward(d_apple_box_price)
print('역전파 수행 후 각 변수의 변화량 값')
print('사과 박스 가격:', d_apple_box)
print('사과 박스 개수:', d_apple_box_num)
print('소비자세:', d_tax)


순전파 수행 후 지불해야 할 최종 금액: 220.00000000000003

역전파 수행 후 각 변수의 변화량 값
사과 박스 가격: 2.2
사과 박스 개수: 110.00000000000001
소비자세: 200


In [4]:
class AddLayer:  # 덧셈 계층
    def __init__(self):
        pass
    
    def forward(self, x, y):
        return x + y # 덧셈계층 계산식 z = x + y
    
    def backward(self, d_out):
        dx = d_out * 1
        dy = d_out * 1
        return dx, dy

# 국소적 계산관점에서 해석하기
# x,y에 대한 z 미분은 1이기에 이전 노드의 미분값을 그대로 넘기는 것이 포인트


## Vanishing Gradient 기울기소실
딥러닝에서 학습은 기울기가 작아지는 방향으로 업데이트를 반복하는 과정이다  
시그모이드같은 활성화 함수의 경우에는 미분값이 작기 때문에   
기울기가 실제 데이터 입력값에 비해 은닉층이나 출력층의 출력값이 작게 나올 수 있다  
이 과정에서 작아진 기울기는 소실된 만큼 학습 효율을 저하시키기 때문에 문제였다  

# 경사하강법  
훈련시 사용되는 전체 데이터를 미분하여 기울기가 낮은 쪽으로 계속 이동시켜 극값을 구하면 이것을 예측값으로 확정하는 원리  
적은 데이터 사용으로 더 빠르게 계산
  
- 확률적 경사하강법(Stochastic Gradient Descent, SGD)  
한 번 업데이트 할 때마다 전체 데이터를 미분하여 진행속도가 느린 단점을 보완하기위해  
랜덤추출한 데이터를 사용하는 확률적 경사하강법을 사용하기도 함  
  
- 모멘텀(Momentum)  
모멘텀은 확률적 경사하강법에서 오차를 수정할 때 수정값과 방향을 참고하는 방법  
관성 개념을 활용하여 로컬 미니멈에 빠져 기존 경사하강법으로는 기울기가 0이라서 계산이 끝났을 상황에서도   
관성으로 값을 조절하여 로컬 미니멈을 넘을 수 있는 효과를 준다  



In [5]:
class SGD:

    def __init__(self, lr=0.01):
        self.lr = lr
        
    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key] 

In [6]:
class Momentum:

    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None
        
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():                                
                self.v[key] = np.zeros_like(val)
                
        for key in params.keys():
            self.v[key] = self.momentum*self.v[key] - self.lr*grads[key] 
            params[key] += self.v[key]